# The Snapshot

This notebook sheds some light on the snapshot of instruments.

In [ ]:
from pprint import pprint
import json
import os

import qcodes as qc

from qcodes.tests.instrument_mocks import DummyInstrument

# For this tutorial, we initialise our favourite pair of mock instruments,
# a DMM and a DAC

dmm = DummyInstrument('dmm', gates=['v1', 'v2'])
dac = DummyInstrument('dac', gates=['ch1', 'ch2'])

station = qc.Station(dmm, dac)

The main point of having a `Station` is that it *snapshots* the state of all added instruments. But what does that mean? Recall that an instrument is, loosely speaking, a collection of `Parameters`.

## Parameter snapshot

In [9]:
# Each parameter has a snapshot, containing information about its current value,
# when that value was set, what the allowed values are, etc.
pprint(dmm.v1.snapshot())

{'__class__': 'qcodes.instrument.parameter.Parameter',
 'full_name': 'dmm_v1',
 'instrument': 'qcodes.tests.instrument_mocks.DummyInstrument',
 'instrument_name': 'dmm',
 'inter_delay': 0,
 'label': 'Gate v1',
 'name': 'v1',
 'post_delay': 0,
 'raw_value': 0,
 'ts': '2017-12-14 16:55:32',
 'unit': 'V',
 'vals': '<Numbers -800<=v<=400>',
 'value': 0}


## Instrument snapshot

In [8]:
# Each instrument has a snapshot that is basically the snapshots of all the parameters
pprint(dmm.snapshot())

{'__class__': 'qcodes.tests.instrument_mocks.DummyInstrument',
 'functions': {},
 'name': 'dmm',
 'parameters': {'IDN': {'__class__': 'qcodes.instrument.parameter.Parameter',
                        'full_name': 'dmm_IDN',
                        'instrument': 'qcodes.tests.instrument_mocks.DummyInstrument',
                        'instrument_name': 'dmm',
                        'inter_delay': 0,
                        'label': 'IDN',
                        'name': 'IDN',
                        'post_delay': 0,
                        'raw_value': {'firmware': None,
                                      'model': 'dmm',
                                      'serial': None,
                                      'vendor': None},
                        'ts': '2017-12-14 16:55:32',
                        'unit': '',
                        'vals': '<Anything>',
                        'value': {'firmware': None,
                                  'model': 'dmm',
                      

## Sweep snapshot

In [10]:
# When running QCoDeS loops, something is being swept. This is controlled with the `sweep` of a parameter.
# Sweeps also have snapshots
a_sweep = dac.ch1.sweep(0, 10, num=25)
pprint(a_sweep.snapshot())

{'parameter': {'__class__': 'qcodes.instrument.parameter.Parameter',
               'full_name': 'dac_ch1',
               'instrument': 'qcodes.tests.instrument_mocks.DummyInstrument',
               'instrument_name': 'dac',
               'inter_delay': 0,
               'label': 'Gate ch1',
               'name': 'ch1',
               'post_delay': 0,
               'raw_value': 0,
               'ts': '2017-12-14 16:55:32',
               'unit': 'V',
               'vals': '<Numbers -800<=v<=400>',
               'value': 0},
 'values': [{'first': 0.0, 'last': 10.0, 'num': 25, 'type': 'linear'}]}


## Loop/Measurement snapshot

In [11]:
# All this is of course nice since a snapshot is saved every time a measurement is 
# performed. Let's see this in action with a Loop.

# This is a qc.loop, sweeping a dac gate and reading a dmm voltage
lp = qc.Loop(dac.ch1.sweep(0, 1, num=10), 0).each(dmm.v1)

# before the loop runs, the snapshot is quite modest; it contains the snapshots of
# the two involved parameters and the sweep
pprint(lp.snapshot())

{'__class__': 'qcodes.loops.ActiveLoop',
 'actions': [{'__class__': 'qcodes.instrument.parameter.Parameter',
   'full_name': 'dmm_v1',
   'instrument': 'qcodes.tests.instrument_mocks.DummyInstrument',
   'instrument_name': 'dmm',
   'inter_delay': 0,
   'label': 'Gate v1',
   'name': 'v1',
   'post_delay': 0,
   'raw_value': 0,
   'ts': '2017-12-14 16:55:32',
   'unit': 'V',
   'vals': '<Numbers -800<=v<=400>',
   'value': 0}],
 'delay': 0,
 'sweep_values': {'parameter': {'__class__': 'qcodes.instrument.parameter.Parameter',
   'full_name': 'dac_ch1',
   'instrument': 'qcodes.tests.instrument_mocks.DummyInstrument',
   'instrument_name': 'dac',
   'inter_delay': 0,
   'label': 'Gate ch1',
   'name': 'ch1',
   'post_delay': 0,
   'raw_value': 0,
   'ts': '2017-12-14 16:55:32',
   'unit': 'V',
   'vals': '<Numbers -800<=v<=400>',
   'value': 0},
  'values': [{'first': 0.0, 'last': 1.0, 'num': 10, 'type': 'linear'}]},
 'then_actions': []}

In [12]:
# After the loop has run, the dataset contains more information, in particular the 
# snapshots for ALL parameters off ALL instruments in the station
data = lp.run('data/dataset')
pprint(data.snapshot())

# This is the snapshot that get's saved to disk alongside your data. 
# It's worthwhile familiarising yourself with it, so that you may retrieve
# valuable information down the line!

Started at 2017-12-14 17:07:16
DataSet:
   location = 'data/2017-12-14/#001_{name}_17-07-16'
   <Type>   | <array_id>  | <array.name> | <array.shape>
   Setpoint | dac_ch1_set | ch1          | (10,)
   Measured | dmm_v1      | v1           | (10,)
Finished at 2017-12-14 17:07:16
{'__class__': 'qcodes.data.data_set.DataSet',
 'arrays': {'dac_ch1_set': {'__class__': 'qcodes.data.data_array.DataArray',
                            'action_indices': (),
                            'array_id': 'dac_ch1_set',
                            'full_name': 'dac_ch1',
                            'instrument': 'qcodes.tests.instrument_mocks.DummyInstrument',
                            'instrument_name': 'dac',
                            'inter_delay': 0,
                            'is_setpoint': True,
                            'label': 'Gate ch1',
                            'name': 'ch1',
                            'post_delay': 0,
                            'raw_value': 0,
                   